In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

In [3]:
train_df = pd.read_csv("../input/students-drop-out-prediction/train.csv")

In [4]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [5]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
        ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [6]:
transform_train_df1 = transform_train_df.copy()
y_train_com = transform_train_df1.pop("label")
X_train_com = transform_train_df1
X_train, X_valid, y_train, y_valid = train_test_split(X_train_com, y_train_com, test_size=0.3, random_state=1)

In [7]:
pca = PCA(random_state=1)
reduced_transform_train_df = pca.fit_transform(transform_train_df[non_binary_col_names])

In [8]:
pca.explained_variance_ratio_[:4]

array([0.25293787, 0.07293534, 0.0653788 , 0.05730171])

In [9]:
reduced_transform_train_df = pd.DataFrame(data = reduced_transform_train_df[:,:4], columns = ["PC1", "PC2", "PC3", "PC4"])
for col in binary_col_names:
    reduced_transform_train_df[col] = transform_train_df[col]
reduced_X_train_com = reduced_transform_train_df.copy()
reduced_y_train_com = y_train_com
reduced_X_train, reduced_X_valid, reduced_y_train, reduced_y_valid = train_test_split(reduced_X_train_com, reduced_y_train_com, test_size=0.3, random_state=1)

In [10]:
# Without PCA
model = DecisionTreeClassifier(class_weight="balanced", random_state=1, ccp_alpha=0.001, max_depth=8)
cross_val_score(model, X_train_com, y_train_com, cv=5, scoring="f1_macro").mean()

0.6265817157765987

In [11]:
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.73      0.59      0.65       375
         1.0       0.80      0.69      0.74       540
         2.0       0.36      0.59      0.44       224

    accuracy                           0.64      1139
   macro avg       0.63      0.62      0.61      1139
weighted avg       0.69      0.64      0.65      1139



In [12]:
model.tree_.max_depth

8

In [13]:
# Grid search without PCA
param_grid = {
    "ccp_alpha": [0.0001, 0.001, 0.01],
    "max_depth": [5, 10, 15],
    "min_samples_split": [5, 10, 15],
    "min_samples_leaf": [5, 10, 15],
    "max_features": ["auto", "log2", None]
}
model = DecisionTreeClassifier(class_weight="balanced", random_state=1)
search = GridSearchCV(model, param_grid=param_grid, scoring="f1_macro", refit=True, cv=5)
search.fit(X_train_com, y_train_com)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             param_grid={'ccp_alpha': [0.0001, 0.001, 0.01],
                         'max_depth': [5, 10, 15],
                         'max_features': ['auto', 'log2', None],
                         'min_samples_leaf': [5, 10, 15],
                         'min_samples_split': [5, 10, 15]},
             scoring='f1_macro')

In [14]:
search.best_params_

{'ccp_alpha': 0.001,
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_leaf': 10,
 'min_samples_split': 5}

In [15]:
search.best_score_

0.6161744838830125

In [16]:
# With PCA
model = DecisionTreeClassifier(class_weight="balanced", random_state=1, max_depth=5, ccp_alpha=0.001)
cross_val_score(model, reduced_X_train_com, reduced_y_train_com, cv=5, scoring="f1_macro").mean()

0.5518006178930926

In [17]:
model.fit(reduced_X_train, reduced_y_train)
preds = model.predict(reduced_X_valid)
print(classification_report(reduced_y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.78      0.61      0.69       375
         1.0       0.76      0.56      0.65       540
         2.0       0.28      0.54      0.36       224

    accuracy                           0.58      1139
   macro avg       0.60      0.57      0.57      1139
weighted avg       0.67      0.58      0.60      1139



In [18]:
# Grid search with PCA
param_grid = {
    "ccp_alpha": [0.0001, 0.001, 0.01],
    "max_depth": [5, 10, 15],
    "min_samples_split": [5, 10, 15],
    "min_samples_leaf": [5, 10, 15],
    "max_features": ["auto", "log2", None]
}
model = DecisionTreeClassifier(class_weight="balanced", random_state=1)
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
search.fit(reduced_X_train_com, reduced_y_train_com)

GridSearchCV(cv=3,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             param_grid={'ccp_alpha': [0.0001, 0.001, 0.01],
                         'max_depth': [5, 10, 15],
                         'max_features': ['auto', 'log2', None],
                         'min_samples_leaf': [5, 10, 15],
                         'min_samples_split': [5, 10, 15]},
             scoring='f1_macro')

In [19]:
search.best_score_

0.5616116223745732

In [20]:
search.best_params_

{'ccp_alpha': 0.001,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 5}

#### Best parameters after HPT
#### ccp_alpha: 0.001
#### max_depth=10
#### max_features=None
#### min_samples_leaf=10
#### min_samples_split=5
#### random_state=1
#### class_weight=balanced
#### CV=5, f1 macro score = 0.616
#### Note that PCA did not help

In [21]:
model = DecisionTreeClassifier(
    class_weight="balanced",
    random_state=1,
    ccp_alpha=0.001,
    max_depth=10,
    max_features=None,
    min_samples_leaf=10,
    min_samples_split=5
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.70      0.63      0.66       375
         1.0       0.79      0.66      0.72       540
         2.0       0.35      0.55      0.43       224

    accuracy                           0.63      1139
   macro avg       0.61      0.61      0.60      1139
weighted avg       0.68      0.63      0.64      1139



In [22]:
# We do not need 'id' column
# Scale non binary features
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")
transform_test_data = transformer.fit_transform(test_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_test_df = pd.DataFrame(data=transform_test_data, columns=feature_names)
preds = [int(i) for i in model.predict(transform_test_df)]
submission = pd.DataFrame({"id": test_df["id"].to_numpy(), "label": preds})
submission.to_csv("submission.csv", index=False)